In [1]:
import requests
import time
import json
import threading
import queue
import random
import datetime
import os
import urllib3

from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup
from collections import defaultdict
from pprint import pprint

from config.assistant import *

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
#from config.site_config import *

In [2]:
# searchType=search
# searchword=컴퓨터비전         # 검색어
# loc_mcd=101000,102000         # 지역, 101000(서울),102000(경기)
# company_cd=0,1,2,3,4,5,6,7    # 채용제공구분 / 9(파견/대행), 10(헤드헌팅)
# search_optional_item=y
# search_done=y
# panel_count=y
# recruitPage=1                 # 현재 페이지뷰
# recruitSort=relation          # 정렬방법
# recruitPageCount=100          # 표시 개수
# inner_com_type=
# show_applied=                 # 지원한 공고 표시
# quick_apply=                  # 빠른 입사지원
# except_read=                  # 조회한 페이지 제외
# ai_head_hunting=
# mainSearch=n

In [3]:
search_word = ["컴퓨터비전", "OPENCV"]
recruit_page = 1
recruit_page_count = 100
company_cd='0,1,2,3,4,5,6,7'

In [4]:
search_link = 'https://www.saramin.co.kr/zf_user/search/recruit\
?search_area=main\
&search_done=y\
&search_optional_item=n\
&searchType=search\
&recruitSort=relation\
&searchword={}\
&recruitPage={}\
&recruitPageCount={}\
&company_cd={}\
&mainSearch=y'.format(search_word[0], recruit_page, recruit_page_count, company_cd)
search_link

'https://www.saramin.co.kr/zf_user/search/recruit?search_area=main&search_done=y&search_optional_item=n&searchType=search&recruitSort=relation&searchword=컴퓨터비전&recruitPage=1&recruitPageCount=100&company_cd=0,1,2,3,4,5,6,7&mainSearch=y'

In [5]:
response = requests.get(search_link, verify=False)
html = response.text
soup = BeautifulSoup(html, 'html.parser')

In [6]:
base_link = 'https://www.saramin.co.kr'
tic()

for i in range(1, recruit_page_count+1):
    elements = soup.select(f'div.content > div:nth-child({i})')[0]

    str_title = elements.find('a', attrs={'class': 'data_layer'})['title']

    str_company = elements.find('a', attrs={'class': 'track_event data_layer'}).string
    str_company = str_company.replace(' ', '').replace('\n', '').replace('(주)', '')

    str_title_link = base_link + elements.find('a', attrs={'class': 'data_layer'})['href']

    str_company_link = base_link + elements.find('a', attrs={'class': 'track_event data_layer'})['href']

toc()



Elapsed time is 4.375982284545898 seconds.


In [13]:
job_group = {}

In [14]:
job_company = "회사_3"
job_title = "제목_4"
title_link = '타이틀링크_1'
company_link = '회사링크_1'

In [15]:
if job_company in job_group:    # 기존
    b_exist = False
    for company in job_group[job_company]:
        if company['title'] == job_title:
            b_exist = True
            print("exist")
            break
    if b_exist == True:
        pass
    else:
        print("additional entry")
        job_group[job_company].append({'title': job_title, \
                                       'title_link': title_link, \
                                       'company_link': company_link \
                                      })
    
else:                           # 신규진입
    print("new entry")
    job_group[job_company] = []
    job_group[job_company].append({'title': job_title, \
                                   'title_link': title_link, \
                                   'company_link': company_link \
                                  })


new entry
